# Download data from Stooq website and store in an efficient way

Read the downloaded data package from stooq and stock as a HDF5. HDF5 has the fastest write/read speed

In [1]:
# import os
# import pandas as pd
# import datetime

# # Specify the directory you want to go through
# directory = 'data'

# # Initialize an empty DataFrame for storing all data
# all_data = pd.DataFrame()

# # Loop through the files in the directory
# for root, dirs, files in os.walk(directory):
#     for filename in files:
#         if filename.endswith('.txt'):
#             filepath = os.path.join(root, filename)
#             try:
#                 # Adjust the 'sep' parameter based on your file's delimiter
#                 temp_df = pd.read_csv(filepath)  # Assuming comma-separated for illustration
                
#                 # Check if the DataFrame is not empty
#                 if not temp_df.empty:
#                     # Extract relative path as folder name for the MultiIndex
#                     folder_name = os.path.relpath(root, directory)
#                     # Create a MultiIndex from the folder name and filename
#                     temp_df.index = pd.MultiIndex.from_product([[folder_name], [filename], range(len(temp_df))],
#                                                                names=['Folder', 'File', 'Row'])
#                     # Append to the main DataFrame
#                     all_data = pd.concat([all_data, temp_df])
#             except Exception as e:
#                 print(f"Failed to read file {filepath}: {e}")

# # Now `all_data` contains all the data with a MultiIndex (Folder, File, Row)
# print(all_data)
# with pd.HDFStore(DATA_STORE) as store:
#     store.put('stooq', all_data)

In [2]:
from pathlib import Path
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
DATA_STORE = Path('stooq.h5')

/Users/ljiehong/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
#load the HDF5 file 
with pd.HDFStore('stooq.h5') as store:
    prices = store['stooq']


## Data Cleaning

In [4]:
prices = prices.reset_index('Row')
prices = prices.set_index(pd.to_datetime(prices['<DATE>'].astype(str)), append = True)
prices = prices.set_index(prices['<TICKER>'], append = True)
prices = prices.drop(['Row', '<PER>', '<TIME>', '<OPENINT>', '<TICKER>', '<DATE>'],axis = 1) #delete the unnecessary columns

In [5]:
# Some adjustment for the dataset, the downloaded file has subdirectory under main directory, delete the subdirectory
dfs = {name: group for name, group in prices.groupby(level=0)}
dfs.keys()
prefix_groups = {}
for key in dfs.keys():
    # Extract the first three directories as the common prefix
    prefix = '/'.join(key.split('/')[:3])
    if prefix not in prefix_groups:
        prefix_groups[prefix] = []
    prefix_groups[prefix].append(key)

combined_dfs = {}
for prefix, keys in prefix_groups.items():
    # Combine all DataFrames within the same group
    combined_dfs[prefix] = pd.concat([dfs[key] for key in keys])


In [6]:
combined_dfs.keys()

dict_keys(['daily/us/nasdaq etfs', 'daily/us/nasdaq stocks', 'daily/us/nyse etfs', 'daily/us/nyse stocks', 'daily/us/nysemkt stocks'])

In [7]:
#Store the dictionary separately as HDF5 file

for i in combined_dfs.keys():
    combined_dfs[i] = combined_dfs[i].reset_index('Folder')
    combined_dfs[i] = combined_dfs[i].drop('Folder', axis = 1)
    combined_dfs[i] = combined_dfs[i].reset_index('File')
    combined_dfs[i] = combined_dfs[i].drop('File', axis = 1)
    
    with pd.HDFStore(DATA_STORE) as store:
        store.put('stooq/'+i, combined_dfs[i])
        
    

In [11]:
#Here is an example for splited dataframe
with pd.HDFStore('stooq.h5') as store:
    temp = store['stooq/daily/us/nasdaq stocks']
temp

,,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
<DATE>,<TICKER>,,,,,
2021-05-18,GTX.US,6.60,6.750,6.3900,6.43,696055.0
2021-05-19,GTX.US,6.42,6.420,6.1500,6.20,383614.0
2021-05-20,GTX.US,6.41,6.410,6.1821,6.30,888296.0
2021-05-21,GTX.US,6.59,6.690,6.3000,6.64,1065179.0
2021-05-24,GTX.US,7.10,7.640,6.9000,7.42,2987620.0
...,...,...,...,...,...,...
2024-02-13,XPEL.US,52.87,54.350,51.7800,52.03,139060.0
2024-02-14,XPEL.US,52.83,53.390,52.0400,53.12,83478.0
2024-02-15,XPEL.US,53.76,56.550,53.6400,56.34,129336.0
